# Задача 3. Сравнение методов классификации

## Список задач
1. Самостоятельно реализовать один из методов классификации, с возможностью настройки гиперпараметров.
2. Взять данные для предсказания заболеваний сердца.
3. Считать данные, выполнить первичный анализ данных, при необходимости произвести чистку данных (Data Cleaning).
4. Выполнить разведочный анализ (EDA), использовать визуализацию, сделать выводы, которые могут быть полезны при дальнейшем решении задачи классификации.
5. При необходимости выполнить полезные преобразования данных (например, трансформировать категариальные признаки в количественные), убрать ненужные признаки, создать новые (Feature Engineering).
6. Используя подбор гиперпараметров, кросс-валидацию и при необходимости масштабирование данных, добиться наилучшего качества предсказания от Вашей реализации на выделенной заранее тестовой выборке.
7. Повторить предыдущий пункт для библиотечных реализаций (например, из sklearn) всех пройденных методов классификации (logistic regression, svm, knn, naive bayes, decision tree).
8. Сравнить все обученные модели, построить их confusion matrices. Сделать выводы о полученных моделях в рамках решения задачи классификации на выбранных данных.
9. Реализовать еще один из методов классификации и добавить его в сравнение.
10. Найти данные, на которых интересно будет решать задачу классификации. Повторить все пункты задания на новых данных.

In [ ]:
import numpy as np
from numpy.typing import NDArray

import polars as pl

# 1/9. Самостоятельно реализовать два метода классификации, с возможностью настройки гиперпараметров.
Реализую дерево решений (Decision Tree) и метод опорных векторов (Support Vector Machine)

In [ ]:
def convert_str_or_float(value: str):
    if(value.isdigit()):
        value = np.float64(value)
    return value


class Metrics:
    def __init__():
        pass
    
    @staticmethod
    def gini(y: NDArray[np.int_]) -> float:
        """
        Вычисляем коэффициент Gini для вектора целевых переменных

        Параметры:
            y (NDArray[np.int_]): Одномерный массив (целочисленный вектор)

        Возвращает:
            float: Значение коэффициента Gini
        """
        count_targets = len(y)

        if count_targets == 0:
            return 0

        _, count_each_target = np.unique(y, return_counts=True)
        probabilities = count_each_target/count_targets
        return 1 - np.sum(probabilities ** 2)

    @staticmethod
    def gini_split(left_y: NDArray[np.float64], right_y: NDArray[np.int_]) -> float:
        """
        
        """
        score = np.inf

        count_objects = len(left_y) + len(right_y)

        left_leaf_loss = Metrics.gini(left_y) 
        rigth_leaf_loss = Metrics.gini(right_y)
        score = ((len(left_y)/count_objects) * left_leaf_loss) + \
                ((len(right_y)/count_objects) * rigth_leaf_loss)

        return score

class Node:
    """
    
    """

    def __init__(self, targets: np.array[np.int_] = None):
        self.left_node = None
        self.right_node = None

        self.targets = targets

    def create_child_nodes(self, X:pl.DataFrame, y:NDArray[np.int_], metric = 'Gini'):
        self.__choose_threshold_feature(X, y, metric)

        left_mask = X[self.feature] <= self.threshold
        right_mask = ~left_mask
        
        y_left = y[left_mask]
        y_right = y[right_mask]

        self.left_node = Node(y_left)
        self.right_node = Node(y_right)

    def __choose_threshold_feature(self, X:pl.DataFrame, y:NDArray[np.int_], metric = 'Gini'):
        """

        """
        best_feature = ''
        best_threshold = ''
        best_loss = np.inf

        features = list(X.columns)
        for feature in features:
            threshold, loss = self.__calculate_threshold_with_metric(X[feature].to_numpy(), y, metric)
            if (loss < best_loss):
                best_feature = feature
                best_threshold = threshold
                best_loss = loss
        
        self.feature = best_feature
        self.threshold = best_threshold

    def __calculate_threshold_with_metric(self, X_column, y, metric):
        """
        Вычисляет оптимальный порог для одного признака X_column на основе метрики.

        Параметры:
            X_column (np.ndarray): Вектор значений признака (n_samples,).
            y (np.ndarray): Вектор целевых переменных (n_samples,).
            metric (str or callable): Выбранная метрика ('Gini', 'Entropy', 'F1', ...) или функция.

        Возвращает:
            tuple: (лучший порог, значение метрики)
        """
        best_threshold = None
        best_score = np.inf

        sorted_indices = np.argsort(X_column)
        X_sorted = X_column[sorted_indices]
        y_sorted = y[sorted_indices]

        for i in range(1, len(X_sorted)):
            left_indices = slice(0, i)
            right_indices = slice(i, len(X_sorted))

            left_y = y_sorted[left_indices]
            right_y = y_sorted[right_indices]

            if metric == 'Gini':
                score = Metrics.gini_split(left_y, right_y)
            elif callable(metric):
                score = metric(left_y, right_y)
            else:
                raise ValueError(f"Неизвестная метрика: {metric}")

            if score < best_score:
                best_score = score
                best_threshold = X_sorted[i-1]

        return best_threshold, best_score

    def majority_class(self):
        """
        Возвращает класс большинства в листе.
        """
        values, counts = np.unique(self.targets, return_counts=True)
        return values[np.argmax(counts)]

class DecisionTree:
    """
    
    """
    
    def __init__(self, depth=3, metric='Gini'):
        self.depth = depth
        self.metric = metric
    
    def code_categorical_features(self, X: pl.DatFrame, y: NDArray[np.int_]):
        self.dict_coding_categorical = []

        categorical_features = [feature for feature, dtype in X.schema.items() if dtype == pl.Utf8]

        if len(categorical_features) == 0:
            return X
        
        for feature in categorical_features:
            unique_values, counts = np.unique(X[feature].to_numpy(), return_counts=True)
            decode_dict = {value: count for value, count in zip(unique_values, counts)}

            X = X.with_columns(
                X[feature].map_dict(decode_dict).alias(feature)
            )

            self.dict_coding_categorical.append({feature: decode_dict})

        return X

    def fit(self, X: pl.DataFrame, y: pl.DataFrame):
        X = self.code_categorical_features(X, y)
        self.root = Node(y)
        last_level_nodes = [self.root]

        for current_depth in range(1, self.depth + 1):
            next_level_nodes = []
            for node in last_level_nodes:
                if node.targets is None or len(node.targets) <= 1:
                    continue
                node.create_child_nodes(X, y, self.metric)
                next_level_nodes.extend([node.left_node, node.right_node])
            last_level_nodes = next_level_nodes

        return self
    
    def predict(self, X: pl.DataFrame):
        y_pred = []
        
        for row in range(X.shape[0]):
            current_node = self.root
            while current_node is not None:
                value = X[row, current_node.feature]

                if value < current_node.threshold:
                    current_node = current_node.left_node
                else:
                    current_node = current_node.right_node
            y_pred.append(current_node.majority_class())

        return y_pred


    def pruning(self):
        pass